## 6. Tuning Your Retrieval Mechanisms

You're about to explore how we can enhance the way we retrieve relevant patient data from a medical dataset using various techniques. We will guide you through running a basic vector search in InterSystems IRIS, and refining results with advanced methods like weighted scoring and hybrid search. Follow along with the steps below, and we'll analyze the outputs together to see how each technique improves efficient retrieval.

### InterSystems IRIS Vector Search: An Overview
InterSystems IRIS Vector Search allows us to store and query high-dimensional vector embeddings within a relational database. These embeddings represent unstructured data like clinical notes as numerical vectors, enabling semantic similarity comparisons. This means we can find patient encounters that are contextually similar to a query, using SQL operations enhanced by VECTORs.By integrating these capabilities into standard SQL operations, IRIS transforms your relational database into a high-performance hybrid vector database—ready to support next-generation AI applications.

### Step 1: Connecting to InterSystems IRIS and Viewing the Dataset
Let's start by connecting to the InterSystems IRIS database to access our medical dataset. Run the code block provided in the notebook to establish this connection and display a snippet of the data.

In this section for this workshop, there is a set of medical data that will be used for experimentation. The data set includes ~1,500 patient encounters, each with structured and coded medical data. With each encounter, however, is also a generated clinical summary note that provides more context about the patient. This might include things such as their commuting situation, their mood during the encounter, or other information not easily categorized into a structured encounter record.

Run the block of code below to initiate a connection to InterSystems IRIS and view a snippet of this data set.

In [ ]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv(override=True)

username = '_SYSTEM'
password = 'SYS'
hostname = 'IRIS'
port = 1972
namespace = 'IRISAPP'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
engine = create_engine(CONNECTION_STRING)

df = pd.read_sql("SELECT * FROM GenAI.Encounters", engine)
df.head()

Notice that in addition to structured data—such as codes, costs, and standardized descriptions of the encounters—there are also columns with unstructured observations and notes, and accompanying vector embeddings. These vector embeddings will help a generative AI application retrieve relevant chunks of data from this set of patient encounters.



### Step 2: Running a Simple Vector Search
In this step, we’ll execute a basic vector search to find patient encounters similar to a query input.

A simple vector search demonstrates how InterSystems IRIS compares embeddings to find semantically relevant results. For example, querying for something like "Headache" would return encounters with similar clinical notes, even if the exact words differ. This is the starting point for understanding retrieval based on meaning rather than keywords

First, run the following line of code to select the sentence transformer model that will be used to create an embedding from your search term. The embedding model you use to embed your search queries should be compatible with the model used to create embeddings in your data set.

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Run the next module, optionally replacing "Headache" with a search term of your choice. This module will create and print an embedding for the search term you have entered.

In [ ]:
note_search = "Headache"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector
print(search_vector)

Now let's run a vector search against our CLINICAL_NOTES field using your search term. With the code below, you will retrieve the top three chunks from the CLINICAL_NOTES field in your data set that are deemed most similar to the search term you provided. The results will be displayed in a Pandas DataFrame for easy viewing.

In [ ]:
from sqlalchemy import text

vector_str = ",".join(str(x) for x in search_vector)
print(vector_str)

with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 ENCOUNTER_ID, CLINICAL_NOTES
            FROM GenAI.encounters
            ORDER BY VECTOR_DOT_PRODUCT(CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector)) DESC
        """)
        results = conn.execute(sql, {"search_vector": vector_str}).fetchall()

# Display results
df = pd.DataFrame(results)
pd.set_option("display.max_colwidth", None)
df.head(10)

### Step 3: Searching across multiple vectorized fields
Let's now consider that you may want to search across more than just your CLINICAL_NOTES field. To improve relevance, we’ll break down similarity across different data categories (notes, observations, etc.) to see which aspects contribute most to the match. This will help us fine-tune our search.

In the block below, you will notice that similarities are being calculated between your search term and all five vectorized fields in the data set. Then, the results are being ordered by the greatest similarity match.

In the result set that follows, explore the similarity scores provided. Sometimes one field provides a particularly good match, while others do not.

Enter whatever search term you would like in the note_search variable. Feel free to play around with multiple searches.

In [ ]:
note_search = "Pregnancy complications"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector
print(search_vector)

vector_str = ",".join(str(x) for x in search_vector) 

with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 5
                ENCOUNTER_ID,
                CLINICAL_NOTES,
                DESCRIPTION_OBSERVATIONS,
                DESCRIPTION_CONDITIONS,
                DESCRIPTION_PROCEDURES,
                DESCRIPTION_MEDICATIONS,
                VECTOR_DOT_PRODUCT (CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector))
                    AS sim_notes,
                VECTOR_DOT_PRODUCT(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector))
                    AS sim_obs,
                VECTOR_DOT_PRODUCT(DESCRIPTION_CONDITIONS_Vector,   TO_VECTOR(:search_vector))
                    AS sim_cond,
                VECTOR_DOT_PRODUCT(DESCRIPTION_PROCEDURES_Vector,   TO_VECTOR(:search_vector))
                    AS sim_proc,
                VECTOR_DOT_PRODUCT(DESCRIPTION_MEDICATIONS_Vector,  TO_VECTOR(:search_vector))
                    AS sim_med
            FROM GenAI.encounters
            ORDER BY GREATEST(
                VECTOR_DOT_PRODUCT(CLINICAL_NOTES_Vector,           TO_VECTOR(:search_vector)),
                VECTOR_DOT_PRODUCT(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)),
                VECTOR_DOT_PRODUCT(DESCRIPTION_CONDITIONS_Vector,   TO_VECTOR(:search_vector)),
                VECTOR_DOT_PRODUCT(DESCRIPTION_PROCEDURES_Vector,   TO_VECTOR(:search_vector)),
                VECTOR_DOT_PRODUCT(DESCRIPTION_MEDICATIONS_Vector,  TO_VECTOR(:search_vector))
) DESC

        """)
        results = conn.execute(sql, {"search_vector": vector_str}).fetchall()
df = pd.DataFrame(results, columns=[
    "ENCOUNTER_ID",
    "CLINICAL_NOTES", "DESCRIPTION_OBSERVATIONS", "DESCRIPTION_CONDITIONS",
    "DESCRIPTION_PROCEDURES", "DESCRIPTION_MEDICATIONS",
    "sim_notes",
    "sim_obs",
    "sim_cond",
    "sim_proc",
    "sim_med"
])
df["DESCRIPTION_OBSERVATIONS"] = df["DESCRIPTION_OBSERVATIONS"].str[:250]
df.head(5)


Looking at the results, notice how ENCOUNTER_ID 1260 has a higher sim_proc score (0.235474) compared to other categories, suggesting the procedures described are more similar to the query. Meanwhile, ENCOUNTER_ID 910 shows higher scores across sim_notes, sim_cond, and sim_med (all 0.235365), indicating broader relevance. This breakdown reveals varying contributions from different data types, but it’s not yet a unified ranking .

Diff with VECTOR DOT PRODUCT and COSINE SIMILARITY
Dot Product: Faster computation but sensitive to vector magnitude
Cosine Similarity: Magnitude-independent but slightly more computation

In [ ]:
note_search = "Pregnancy complications"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector
print(search_vector)

vector_str = ",".join(str(x) for x in search_vector) 

with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 5
                ENCOUNTER_ID,
                CLINICAL_NOTES,
                DESCRIPTION_OBSERVATIONS,
                DESCRIPTION_CONDITIONS,
                DESCRIPTION_PROCEDURES,
                DESCRIPTION_MEDICATIONS,
                VECTOR_COSINE (CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector))
                    AS sim_notes,
                VECTOR_COSINE (DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector))
                    AS sim_obs,
                VECTOR_COSINE (DESCRIPTION_CONDITIONS_Vector,   TO_VECTOR(:search_vector))
                    AS sim_cond,
                VECTOR_COSINE (DESCRIPTION_PROCEDURES_Vector,   TO_VECTOR(:search_vector))
                    AS sim_proc,
                VECTOR_COSINE(DESCRIPTION_MEDICATIONS_Vector,  TO_VECTOR(:search_vector))
                    AS sim_med
            FROM GenAI.encounters
            ORDER BY GREATEST(
                VECTOR_COSINE (CLINICAL_NOTES_Vector,           TO_VECTOR(:search_vector)),
                VECTOR_COSINE (DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)),
                VECTOR_COSINE (DESCRIPTION_CONDITIONS_Vector,   TO_VECTOR(:search_vector)),
                VECTOR_COSINE (DESCRIPTION_PROCEDURES_Vector,   TO_VECTOR(:search_vector)),
                VECTOR_COSINE (DESCRIPTION_MEDICATIONS_Vector,  TO_VECTOR(:search_vector))
) DESC

        """)
        results = conn.execute(sql, {"search_vector": vector_str}).fetchall()
df = pd.DataFrame(results, columns=[
    "ENCOUNTER_ID",
    "CLINICAL_NOTES", "DESCRIPTION_OBSERVATIONS", "DESCRIPTION_CONDITIONS",
    "DESCRIPTION_PROCEDURES", "DESCRIPTION_MEDICATIONS",
    "sim_notes",
    "sim_obs",
    "sim_cond",
    "sim_proc",
    "sim_med"
])
df["DESCRIPTION_OBSERVATIONS"] = df["DESCRIPTION_OBSERVATIONS"].str[:250]
df.head(5)


you're noticing that above queries using VECTOR_DOT_PRODUCT and VECTOR_COSINE produce similar results. This is actually expected when working with normalized embeddings.

Why They Produce Similar Results
The key is in the embedding generation:

search_vector = model.encode(note_search, normalize_embeddings=True).tolist()
When you set normalize_embeddings=True, your model is creating unit vectors (vectors with length 1). With normalized vectors:

Cosine Similarity = Dot Product: For unit vectors, the dot product is mathematically equivalent to cosine similarity.
Mathematical Explanation (if needed)
Cosine similarity is defined as: cos(θ) = (A·B)/(|A|·|B|)
When |A| = |B| = 1 (normalized vectors), this simplifies to: cos(θ) = A·B

In hybrid search function later, we will be using cosine similarity with normalized vectors, which is a common best practice for semantic search.



### Implementing Weight Adjustments for Multiple Vector Fields


**Define Weights:** Assign weights to each vector field. For example:

In [ ]:
weights = {
    'sim_notes': 0.25,
    'sim_obs': 0.35,
    'sim_cond': 0.1,
    'sim_proc': 0.2,
    'sim_med': 0.1
} 

**Modify the SQL Query**
Adjust the SQL query to multiply each similarity score by its corresponding weight and sum them up.

In [ ]:
note_search = "diabetes"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist()  # Convert search phrase into a vector
vector_str = ",".join(str(x) for x in search_vector)

with engine.connect() as conn:
    with conn.begin():
        sql = text(f"""
            SELECT TOP 5
                ENCOUNTER_ID,
                CLINICAL_NOTES,
                DESCRIPTION_OBSERVATIONS,
                DESCRIPTION_CONDITIONS,
                DESCRIPTION_PROCEDURES,
                DESCRIPTION_MEDICATIONS,
                sim_notes,
                sim_obs,
                sim_cond,
                sim_proc,
                sim_med,
                (
                    sim_notes * {weights['sim_notes']} +
                    sim_obs * {weights['sim_obs']} +
                    sim_cond * {weights['sim_cond']} +
                    sim_proc * {weights['sim_proc']} +
                    sim_med * {weights['sim_med']}
                ) AS weighted_sim
            FROM (
                SELECT
                    ENCOUNTER_ID,
                    CLINICAL_NOTES,
                    DESCRIPTION_OBSERVATIONS,
                    DESCRIPTION_CONDITIONS,
                    DESCRIPTION_PROCEDURES,
                    DESCRIPTION_MEDICATIONS,
                    VECTOR_DOT_PRODUCT(CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector)) AS sim_notes,
                    VECTOR_DOT_PRODUCT(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)) AS sim_obs,
                    VECTOR_DOT_PRODUCT(DESCRIPTION_CONDITIONS_Vector, TO_VECTOR(:search_vector)) AS sim_cond,
                    VECTOR_DOT_PRODUCT(DESCRIPTION_PROCEDURES_Vector, TO_VECTOR(:search_vector)) AS sim_proc,
                    VECTOR_DOT_PRODUCT(DESCRIPTION_MEDICATIONS_Vector, TO_VECTOR(:search_vector)) AS sim_med
                FROM GenAI.encounters
            ) AS subquery
            ORDER BY weighted_sim DESC
        """)
        results = conn.execute(sql, {"search_vector": vector_str}).fetchall()

df = pd.DataFrame(results, columns=[
    "ENCOUNTER_ID",
    "CLINICAL_NOTES", "DESCRIPTION_OBSERVATIONS", "DESCRIPTION_CONDITIONS",
    "DESCRIPTION_PROCEDURES", "DESCRIPTION_MEDICATIONS",
    "sim_notes",
    "sim_obs",
    "sim_cond",
    "sim_proc",
    "sim_med",
    "weighted_sim"
])
df["DESCRIPTION_OBSERVATIONS"] = df["DESCRIPTION_OBSERVATIONS"].str[:250]
df.head(5)

## Hybrid Search
Combine keyword-based search with vector-based search for more comprehensive results. For example, you can use a text search on the `note` field and combine it with the vector search.
The hybrid search combines the strengths of both vector-based semantic search and traditional keyword search, which can provide more relevant results than either method alone. Vector search is good at understanding semantic meaning, while keyword search can catch exact matches that might be missed by the embedding model.
You can adjust the `vector_weight` and `keyword_weight` parameters to fine-tune the balance between semantic similarity and keyword matching based on your specific use case.

In [ ]:
def hybrid_search(query, engine, model, top_n=5, vector_weight=0.7, keyword_weight=0.3):
    """
    Parameters:
    - query: Search query string
    - engine: SQLAlchemy engine with connection to InterSystems IRIS
    - model: model for embeddings
    - top_n: Number of results to return
    - vector_weight: Weight for vector search results (0.0 to 1.0)
    - keyword_weight: Weight for keyword search results (0.0 to 1.0)
    """
    
    # Step 1: Perform vector search
    search_vector = model.encode(query, normalize_embeddings=True).tolist()
    vector_str = ",".join(str(x) for x in search_vector)
    
    with engine.connect() as conn:
        vector_sql = text(f"""
            SELECT 
                TOP {top_n * 2}
                ENCOUNTER_ID,
                CLINICAL_NOTES,
                DESCRIPTION_OBSERVATIONS,
                DESCRIPTION_CONDITIONS,
                DESCRIPTION_PROCEDURES,
                DESCRIPTION_MEDICATIONS,
                VECTOR_COSINE(CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector)) AS sim_notes,
                VECTOR_COSINE(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)) AS sim_obs,
                VECTOR_COSINE(DESCRIPTION_CONDITIONS_Vector, TO_VECTOR(:search_vector)) AS sim_cond,
                VECTOR_COSINE(DESCRIPTION_PROCEDURES_Vector, TO_VECTOR(:search_vector)) AS sim_proc,
                VECTOR_COSINE(DESCRIPTION_MEDICATIONS_Vector, TO_VECTOR(:search_vector)) AS sim_med,
                (
                    VECTOR_COSINE(CLINICAL_NOTES_Vector, TO_VECTOR(:search_vector)) * {weights['sim_notes']} +
                    VECTOR_COSINE(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)) * {weights['sim_obs']} +
                    VECTOR_COSINE(DESCRIPTION_CONDITIONS_Vector, TO_VECTOR(:search_vector)) * {weights['sim_cond']} +
                    VECTOR_COSINE(DESCRIPTION_PROCEDURES_Vector, TO_VECTOR(:search_vector)) * {weights['sim_proc']} +
                    VECTOR_COSINE(DESCRIPTION_MEDICATIONS_Vector, TO_VECTOR(:search_vector)) * {weights['sim_med']}
                ) AS vector_score
            FROM GenAI.encounters
            ORDER BY vector_score DESC
        """)
        
        vector_results = conn.execute(vector_sql, {"search_vector": vector_str}).fetchall()
        
        # Step 2: Perform keyword search
        # Extract keywords (simple implementation)
        keywords = [word.lower() for word in query.split() if len(word) > 3]
        if not keywords:
            keywords = [query.lower()]
        
        # Build LIKE conditions for each keyword
        like_conditions = []
        for field in ['CLINICAL_NOTES', 'DESCRIPTION_OBSERVATIONS', 'DESCRIPTION_CONDITIONS', 
                     'DESCRIPTION_PROCEDURES', 'DESCRIPTION_MEDICATIONS']:
            for keyword in keywords:
                like_conditions.append(f"{field} LIKE '%{keyword}%'")
        
        where_clause = " OR ".join(like_conditions)
        
        # Keyword search query
        keyword_sql = text(f"""
            SELECT 
                TOP {top_n * 2}
                ENCOUNTER_ID,
                CLINICAL_NOTES,
                DESCRIPTION_OBSERVATIONS,
                DESCRIPTION_CONDITIONS,
                DESCRIPTION_PROCEDURES,
                DESCRIPTION_MEDICATIONS,
                (
                    {" + ".join([f"(CASE WHEN {field} LIKE '%{keyword}%' THEN 1 ELSE 0 END)" 
                                for field in ['CLINICAL_NOTES', 'DESCRIPTION_OBSERVATIONS', 
                                             'DESCRIPTION_CONDITIONS', 'DESCRIPTION_PROCEDURES', 
                                             'DESCRIPTION_MEDICATIONS']
                                for keyword in keywords])}
                ) AS keyword_score
            FROM GenAI.encounters
            WHERE {where_clause}
            ORDER BY keyword_score DESC
        """)
        
        keyword_results = conn.execute(keyword_sql).fetchall()
    
    # Step 3: Convert results to DataFrames
    vector_df = pd.DataFrame(vector_results, columns=[
        "ENCOUNTER_ID", "CLINICAL_NOTES", "DESCRIPTION_OBSERVATIONS", 
        "DESCRIPTION_CONDITIONS", "DESCRIPTION_PROCEDURES", "DESCRIPTION_MEDICATIONS",
        "sim_notes", "sim_obs", "sim_cond", "sim_proc", "sim_med", "vector_score"
    ])
    
    keyword_df = pd.DataFrame(keyword_results, columns=[
        "ENCOUNTER_ID", "CLINICAL_NOTES", "DESCRIPTION_OBSERVATIONS", 
        "DESCRIPTION_CONDITIONS", "DESCRIPTION_PROCEDURES", "DESCRIPTION_MEDICATIONS",
        "keyword_score"
    ])
    
    # Step 4: Normalize scores to 0-1 range
    if not vector_df.empty:
        # UPDATED: Convert string to numeric values
        vector_df['vector_score'] = pd.to_numeric(vector_df['vector_score'], errors='coerce')
        max_vector_score = vector_df['vector_score'].max()
        if pd.notnull(max_vector_score) and max_vector_score > 0:
            vector_df['vector_score_norm'] = vector_df['vector_score'] / max_vector_score
        else:
            vector_df['vector_score_norm'] = vector_df['vector_score']
    
    if not keyword_df.empty:
        # UPDATED: Convert string to numeric values
        keyword_df['keyword_score'] = pd.to_numeric(keyword_df['keyword_score'], errors='coerce')
        max_keyword_score = keyword_df['keyword_score'].max()
        if pd.notnull(max_keyword_score) and max_keyword_score > 0:
            keyword_df['keyword_score_norm'] = keyword_df['keyword_score'] / max_keyword_score
        else:
            keyword_df['keyword_score_norm'] = keyword_df['keyword_score']
    
    # Step 5: Merge results
    # Start with all vector results
    combined_results = vector_df.copy() if not vector_df.empty else pd.DataFrame()

    # Add keyword score column (0 for entries only in vector results)
    if not combined_results.empty:
        combined_results['keyword_score'] = 0
        combined_results['keyword_score_norm'] = 0

    # Add keyword results not already in vector results
    if not keyword_df.empty:
        # Find keyword results not in vector results
        if not combined_results.empty:
            keyword_only = keyword_df[~keyword_df['ENCOUNTER_ID'].isin(combined_results['ENCOUNTER_ID'])].copy()  # Added .copy()
        else:
            keyword_only = keyword_df.copy()  # Added .copy()
            
        # Add vector score columns (0 for entries only in keyword results)
        if not keyword_only.empty:
            # FIX: Use .loc to avoid the SettingWithCopyWarning
            for col in ['sim_notes', 'sim_obs', 'sim_cond', 'sim_proc', 'sim_med', 'vector_score', 'vector_score_norm']:
                keyword_only.loc[:, col] = 0  # Changed to use .loc
            
            # Append to combined results
            combined_results = pd.concat([combined_results, keyword_only])

    # Step 6: Update scores for entries in both result sets
    if not combined_results.empty and not keyword_df.empty:
        # For each row in combined results that also exists in keyword results
        for encounter_id in combined_results['ENCOUNTER_ID']:
            keyword_match = keyword_df[keyword_df['ENCOUNTER_ID'] == encounter_id]
            if not keyword_match.empty:
                # UPDATED: Ensure values are numeric before assignment
                combined_results.loc[combined_results['ENCOUNTER_ID'] == encounter_id, 'keyword_score'] = pd.to_numeric(keyword_match['keyword_score'].values[0], errors='coerce')
                combined_results.loc[combined_results['ENCOUNTER_ID'] == encounter_id, 'keyword_score_norm'] = pd.to_numeric(keyword_match['keyword_score_norm'].values[0], errors='coerce')
    
    # Step 7: Calculate hybrid score
    if not combined_results.empty:
        # UPDATED: Ensure values are numeric before calculation
        combined_results['vector_score_norm'] = pd.to_numeric(combined_results['vector_score_norm'], errors='coerce').fillna(0)
        combined_results['keyword_score_norm'] = pd.to_numeric(combined_results['keyword_score_norm'], errors='coerce').fillna(0)
        
        combined_results['hybrid_score'] = (
            combined_results['vector_score_norm'] * vector_weight + 
            combined_results['keyword_score_norm'] * keyword_weight
        )
        
        # Sort by hybrid score
        combined_results = combined_results.sort_values(by='hybrid_score', ascending=False)
    
    # Step 8: Return top N results
    return combined_results.head(top_n)

In [ ]:
query = "diabetes"
results_df = hybrid_search(
    query=query,
    engine=engine,
    model=model,
    top_n=5,
    vector_weight=0.7,
    keyword_weight=0.3
)

results_df["DESCRIPTION_OBSERVATIONS"] = results_df["DESCRIPTION_OBSERVATIONS"].str[:250]

# Display the top 5 results
print(f"Hybrid Search Results for: '{query}'")
results_df.head(5)

In [ ]:
def test_hybrid_search():
    """Test the hybrid search functionality with a sample query."""
    from dotenv import load_dotenv
    from sqlalchemy import create_engine
    from sentence_transformers import SentenceTransformer
    import os
    from tabulate import tabulate  # Install with: pip install tabulate
    import textwrap
    from colorama import Fore, Style, init  # Install with: pip install colorama
    
    # Initialize colorama for colored terminal output
    init(autoreset=True)
    
   
    
    # Test queries
    test_queries = [
        "diabetes",
        "diabetic"
    ]
    
    for query in test_queries:
        print(f"\n\n{Fore.CYAN}{'=' * 100}")
        print(f"{Fore.CYAN}Hybrid Search Results for: {Fore.YELLOW}'{query}'")
        print(f"{Fore.CYAN}{'=' * 100}")
        
        # Perform hybrid search
        results = hybrid_search(
            query=query,
            engine=engine,
            model=model,
            top_n=5,
            vector_weight=0.7,
            keyword_weight=0.3
        )
        
        if results.empty:
            print(f"{Fore.RED}No results found.")
        else:
            # Format results for display
            display_data = []
            
            for index, row in results.iterrows():
                # Truncate and format the clinical notes
                notes_preview = textwrap.shorten(row['CLINICAL_NOTES'], width=80, placeholder="...")
                
                # Format scores with appropriate precision
                vector_score = f"{float(row['vector_score']):.4f}"
                keyword_score = f"{float(row['keyword_score']):.0f}"
                hybrid_score = f"{float(row['hybrid_score']):.4f}"
                
                display_data.append([
                    row['ENCOUNTER_ID'],
                    notes_preview,
                    vector_score,
                    keyword_score,
                    hybrid_score
                ])
            
            # Display results in a nicely formatted table
            headers = [
                f"{Fore.GREEN}Encounter ID", 
                f"{Fore.GREEN}Clinical Notes Preview", 
                f"{Fore.GREEN}Vector Score", 
                f"{Fore.GREEN}Keyword Score", 
                f"{Fore.GREEN}Hybrid Score"
            ]
            
            print(tabulate(display_data, headers=headers, tablefmt="fancy_grid"))
            
            # Print detailed information for the top result
            if len(results) > 0:
                top_result = results.iloc[0]
                
                print(f"\n{Fore.MAGENTA}Top Result Details:{Style.RESET_ALL}")
                print(f"{Fore.MAGENTA}{'-' * 100}")
                
                # Encounter details
                print(f"{Fore.YELLOW}Encounter ID:{Style.RESET_ALL} {top_result['ENCOUNTER_ID']}")
                print(f"{Fore.YELLOW}Vector Score:{Style.RESET_ALL} {float(top_result['vector_score']):.4f}")
                print(f"{Fore.YELLOW}Keyword Score:{Style.RESET_ALL} {float(top_result['keyword_score']):.0f}")
                print(f"{Fore.YELLOW}Hybrid Score:{Style.RESET_ALL} {float(top_result['hybrid_score']):.4f}")
                
                # Format clinical notes with proper wrapping
                print(f"\n{Fore.YELLOW}Clinical Notes:{Style.RESET_ALL}")
                notes = top_result['CLINICAL_NOTES']
                if len(notes) > 800:
                    notes = notes[:800] + "..."
                
                # Wrap text for better readability
                wrapped_notes = textwrap.fill(notes, width=100)
                print(wrapped_notes)
                
                # Show observations if available
                if top_result['DESCRIPTION_OBSERVATIONS']:
                    print(f"\n{Fore.YELLOW}Observations:{Style.RESET_ALL}")
                    obs = top_result['DESCRIPTION_OBSERVATIONS']
                    if len(obs) > 400:
                        obs = obs[:400] + "..."
                    print(textwrap.fill(obs, width=100))
                
                # Show conditions if available
                if top_result['DESCRIPTION_CONDITIONS']:
                    print(f"\n{Fore.YELLOW}Conditions:{Style.RESET_ALL}")
                    cond = top_result['DESCRIPTION_CONDITIONS']
                    if len(cond) > 400:
                        cond = cond[:400] + "..."
                    print(textwrap.fill(cond, width=100))

# Run the test function
if __name__ == "__main__":
    test_hybrid_search()